In [1]:
import torch

from tqdm import tqdm


from args import DataTrainingArguments, ArgumentParser, PromptVectorConfig
from tasks import AutoTask

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import get_peft_model
from trl import SFTConfig, ModelConfig

from utils import load_prompt

In [2]:
def apply_test_template(examples, tokenizer):
    return {
        "text": tokenizer.apply_chat_template(
            [examples], tokenize=False, add_generation_prompt=True
        )
    }


In [13]:
def predict(test_dataset, model, tokenizer):
    y_pred = []
    pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        do_sample=False,
        top_p=None,
        use_cache=False,
        temperature=None,
        device="cuda",
    )

    for x_test in tqdm(test_dataset["text"]):

        print(x_test)

        result = pipe(x_test)
        answer = (
            result[0]["generated_text"]
            .split("<|eot_id|><|start_header_id|>assistant<|end_header_id|>")[-1]
            .strip()
        )

        print(answer)

        y_pred.append(answer)

    return y_pred


In [4]:
parser = ArgumentParser(
    (SFTConfig, ModelConfig, DataTrainingArguments, PromptVectorConfig)
)

training_args, model_args, data_args, peft_config = parser.parse_toml_file(
    "configs/slovak_alpaca.toml"
)

save_path = "/home/jovyan/language-prompt-vectors/saves/prompt_tuning_10292024113646_slovak_alpaca_origin_0_meta-llama-3.1-8b-instruct/checkpoint-104010/adapter_model.safetensors"

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_args.model_name_or_path,
    torch_dtype=torch.bfloat16,
).to("cuda")
model.active_adapters = [
    "default"
]  # fix because llama has some active adapters for some reason
model = get_peft_model(model, peft_config=peft_config)

tokenizer = AutoTokenizer.from_pretrained(
    data_args.data_tokenizer_name_or_path,
    trust_remote_code=True,
    padding_side="right",
)
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
dataset_name = data_args.dataset_names[0]

test_dataset = AutoTask.get(dataset_name).get(
    split="test",
    task_type=peft_config.task_type,
    add_prefix=False,
    n_obs=data_args.max_test_samples,
    split_validation_test=data_args.split_validation_test,
)

chat_test_dataset = test_dataset.map(apply_test_template, fn_kwargs={"tokenizer": tokenizer})

Running slovak_alpaca_preprocessor on dataset:   0%|          | 0/5201 [00:00<?, ? examples/s]

In [7]:
results = predict(chat_test_dataset[:2], model, tokenizer, AutoTask.get(dataset_name).labels_list)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Vytvorte tabuľku SQL s nasledujúcimi stĺpcami: _id, name, phone, email.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




 50%|█████████████████████████████████████████████████████████████████████████████████████████████                                                                                             | 1/2 [00:07<00:07,  7.35s/it]

Aby vytvoriť tabuľku v SQL, použijeme nasledujúci príkaz:

```sql
CREATE TABLE users (
  _id INT PRIMARY KEY,
  name VARCHAR(255),
  phone VARCHAR(20),
  email VARCHAR(255)
);
```

V tomto príklade:

- `CREATE TABLE` je príkaz, ktorý vytvára novú tabuľku.
- `users` je názov tabuľky.
- `id` je prvý stĺpec, ktorý je definovaný ako `INT` (celečné číslo) a je označený ako `PRIMARY KEY`, čo znamená, že je to jedinečné identifikátor pre každú riadok v tabuľke.
- `name`, `phone` a `email` sú ďalší stĺpce, ktoré sú definované ako `VARCHAR(255)`, čo znamená, že môžu obsahovať maximálne 255 znakových údajov.

Poznámka: V mnohých databázových systémoch (napríklad MySQL) sa používa `id` ako názov pre prvý stĺpec, ale v iných systémoch (napríklad PostgreSQL) sa používa `id` alebo `id SERIAL` pre generovanie automaticky generovaných ID.
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_he

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:19<00:00,  9.73s/it]

Hustota obyvateľstva a rast obyvateľstva sú dve rozličné koncepty, ktoré sa vzťahujú k demografickým údajom o populácii.

**Hustota obyvateľstva** je meradlo, ktoré udáva počet obyvateľov na jednotku plochy. Je to obvykle vyjadrené v jednotkách obyvateľov na km² (obyv./km²). Hustota obyvateľstva sa používa na popis charakteru urbanizácie a rozlohy oblasti. Vyššia hustota obyvateľstva indikuje, že v danej oblasti je veľa ľudí na malom priestore, čo môže byť typické pre mestá alebo aglomerácie.

**Rast obyvateľstva** je meradlo, ktoré udáva zmeny v počte obyvateľov v danej oblasti v čase. Rast obyvateľstva môže byť pozitívny (t.j. rast) alebo negatívny (t.j. pokles). Rast obyvateľstva sa používa na popis demografických trendov a môže byť ovplyvnený rôznymi faktormi, ako napríklad narodeninami, úmrtnosťou, migráciou a demografickými zmenami.

Rozdiel medzi hustotou obyvateľstva a rastom obyvateľstva je nasledovný:

- Hustota obyvateľstva sa týka počtu obyvateľov na jednotku plochy, zatiaľ

In [8]:
print(test_dataset["target"][0])
print(test_dataset["target"][1])

 Tu je príklad príkazu SQL, ktorý možno použiť na vytvorenie tabuľky so stĺpcami _id, name, phone a email: ``` CREATE TABLE contacts ( _id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT NOT NULL, phone TEXT NOT NULL , email TEXT NOT NULL ); ```
Hustota obyvateľstva a rast populácie sú dva odlišné demografické pojmy. Hustota obyvateľstva sa vzťahuje na počet ľudí žijúcich v určitej geografickej oblasti, ako je mesto, štát alebo krajina, v pomere k jej veľkosti. Bežne sa meria ako počet ľudí na jednotku plochy, napríklad na míľu štvorcovú alebo kilometer štvorcový. Toto meranie poskytuje predstavu o tom, ako je oblasť preplnená a koľko miesta je k dispozícii pre každú osobu v tejto oblasti. Na druhej strane, rast populácie sa týka zmeny počtu ľudí žijúcich v určitej oblasti v priebehu času. Môže sa merať ako absolútny nárast alebo pokles alebo ako percentuálna zmena. Toto meranie je ovplyvnené rôznymi faktormi, akými sú miera pôrodnosti a úmrtnosti, migrácia a zmeny vekovej štruktúry obyva

In [10]:
model.prompt_encoder.default.embedding.weight = torch.nn.Parameter(
    load_prompt(save_path).to("cuda")
)

In [14]:
results = predict(chat_test_dataset[:2], model, tokenizer, AutoTask.get(dataset_name).labels_list)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Vytvorte tabuľku SQL s nasledujúcimi stĺpcami: _id, name, phone, email.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




 50%|█████████████████████████████████████████████████████████████████████████████████████████████                                                                                             | 1/2 [00:01<00:01,  1.93s/it]

Tu je príklad tabuľky SQL s uvedenými stĺpcami: ``` SQL CREATE TABLE contacts ( id INT PRIMARY KEY, name VARCHAR(255) NOT NULL, phone VARCHAR(20) NOT NULL, email VARCHAR(255) NOT NULL); ```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Vysvetlite rozdiel medzi hustotou obyvateľstva a rastom obyvateľstva.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.80s/it]

Hustota obyvateľstva a rast obyvateľstva sú dva odlišné pojmy, ktoré sa vzťahujú na demografické údaje o populácii. Hustota obyvateľstva sa meria ako počet obyvateľov na jednotku plochy, zvyčajne vyjadrená ako počet obyvateľov na kilometr štvorcový (km²). Je to miera, ktorá ukazuje, ako husto sú obyvatelia usadení v určitej oblasti. Napríklad krajina s hustotou obyvateľstva 100 obyvateľov na km² má väčšiu populáciu, ako krajina s hustotou obyvateľstva 10 obyvateľov na km², alebo akékoľvek dve krajiny s rovnakou populáciou, krajina s vyššou hustotou obyvateľstva bude mať menšiu plochu. Hustota obyvateľstva sa môže meniť v čase v dôsledku rastu alebo poklesu populácie a zmeny veľkosti územia. Rast obyvateľstva sa meria ako zmena počtu obyvateľov v určitej oblasti v dôsledku narodenia, úmrtia a migrácie. Je to miera, ktorá ukazuje, ako sa populácia zvyšuje alebo znižuje v čase. Napríklad krajina s rastom obyvateľstva 2 % za rok má v priebehu roka 2 % väčšiu populáciu, ako v priebehu roka 